# Playoff LeBron (?)

Let's make some charts that investigate whether 'Playoff LeBron' really exists, and how he differs from his idols and contemporaries alike!

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Load full stats dataset (all players)

In [3]:
stats = pd.read_csv('/Users/TBD/Documents/DevStuff/DataSciProjects/NBAPlayerMovement/Seasons_Stats.csv')

In [4]:
#list(stats.columns.values)

# Remove unnecessary columns
dumb_cols = ["Unnamed: 0", "blanl", "blank2"]

stats.drop(dumb_cols, axis=1, inplace=True)

In [5]:
stats = stats[stats.Year >= 1980]
stats.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
5727,1980.0,Kareem Abdul-Jabbar*,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,...,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0
5728,1980.0,Tom Abernethy,PF,25.0,GSW,67.0,NaN,1222.0,11.0,0.511,...,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0
5729,1980.0,Alvan Adams,C,25.0,PHO,75.0,NaN,2168.0,19.2,0.571,...,0.797,158.0,451.0,609.0,322.0,108.0,55.0,218.0,237.0,1118.0
5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,...,0.830,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0
5731,1980.0,Dennis Awtrey,C,31.0,CHI,26.0,NaN,560.0,7.4,0.524,...,0.640,29.0,86.0,115.0,40.0,12.0,15.0,27.0,66.0,86.0


In [6]:
# Remove those pesky trailing asteric's
for n in stats['Player']:
    try:
        if n.endswith('*'):
            stats['Player'] = stats['Player'].replace(n, n[:-1])
    except:
        continue

In [7]:
stats.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
5727,1980.0,Kareem Abdul-Jabbar,C,32.0,LAL,82.0,NaN,3143.0,25.3,0.639,...,0.765,190.0,696.0,886.0,371.0,81.0,280.0,297.0,216.0,2034.0
5728,1980.0,Tom Abernethy,PF,25.0,GSW,67.0,NaN,1222.0,11.0,0.511,...,0.683,62.0,129.0,191.0,87.0,35.0,12.0,39.0,118.0,362.0
5729,1980.0,Alvan Adams,C,25.0,PHO,75.0,NaN,2168.0,19.2,0.571,...,0.797,158.0,451.0,609.0,322.0,108.0,55.0,218.0,237.0,1118.0
5730,1980.0,Tiny Archibald,PG,31.0,BOS,80.0,80.0,2864.0,15.3,0.574,...,0.830,59.0,138.0,197.0,671.0,106.0,10.0,242.0,218.0,1131.0
5731,1980.0,Dennis Awtrey,C,31.0,CHI,26.0,NaN,560.0,7.4,0.524,...,0.640,29.0,86.0,115.0,40.0,12.0,15.0,27.0,66.0,86.0


### Load individual players playoff stats datasets

In [8]:
# First, load playoff totals
lbj_playoffs_totals = pd.read_csv('datasets/lebron_playoffs_totals.csv')  # LeBron
bird_playoffs_totals = pd.read_csv('datasets/bird_playoffs_totals.csv')   # Larry Bird
magic_playoffs_totals = pd.read_csv('datasets/magic_playoffs_totals.csv') # Magiv Johnson
kobe_playoffs_totals = pd.read_csv('datasets/kobe_playoffs_totals.csv')   # Kobe Bryant
curry_playoffs_totals = pd.read_csv('datasets/curry_playoffs_totals.csv') # Steph Curry
chrispaul_playoffs_totals = pd.read_csv('datasets/chrispaul_playoffs_totals.csv') # Chris Paul
kg_playoffs_totals = pd.read_csv('datasets/kg_playoffs_totals.csv')  # Kevin Garnett
mj_playoffs_totals = pd.read_csv('datasets/mj_playoffs_totals.csv')  # Michael Jordan
harden_playoffs_totals = pd.read_csv('datasets/harden_playoffs_totals.csv')  # James Harden
durant_playoffs_totals = pd.read_csv('datasets/durant_playoffs_totals.csv')  # Kevin Durant
westbrook_playoffs_totals = pd.read_csv('datasets/westbrook_playoffs_totals.csv') # Russell Westbrook
nash_playoffs_totals = pd.read_csv('datasets/nash_playoffs_totals.csv')  # Steve Nash
shaq_playoffs_totals = pd.read_csv('datasets/shaq_playoffs_totals.csv')  # Shaquiel O'Neal

In [7]:
#with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
#    print(lbj_playoffs_totals)

In [9]:
# Next, load playoffs advanced stats
lbj_playoffs_advanced = pd.read_csv('datasets/lebron_playoffs_advanced.csv')  # LeBron
bird_playoffs_advanced = pd.read_csv('datasets/bird_playoffs_advanced.csv')   # Larry Bird
magic_playoffs_advanced = pd.read_csv('datasets/magic_playoffs_advanced.csv') # Magiv Johnson
kobe_playoffs_advanced = pd.read_csv('datasets/kobe_playoffs_advanced.csv')   # Kobe Bryant
curry_playoffs_advanced = pd.read_csv('datasets/curry_playoffs_advanced.csv') # Steph Curry
chrispaul_playoffs_advanced = pd.read_csv('datasets/chrispaul_playoffs_advanced.csv') # Chris Paul
kg_playoffs_advanced = pd.read_csv('datasets/kg_playoffs_advanced.csv')  # Kevin Garnett
mj_playoffs_advanced = pd.read_csv('datasets/mj_playoffs_advanced.csv')  # Michael Jordan
harden_playoffs_advanced = pd.read_csv('datasets/harden_playoffs_advanced.csv')  # James Harden
durant_playoffs_advanced = pd.read_csv('datasets/durant_playoffs_advanced.csv')  # Kevin Durant
westbrook_playoffs_advanced = pd.read_csv('datasets/westbrook_playoffs_advanced.csv') # Russell Westbrook
nash_playoffs_advanced = pd.read_csv('datasets/nash_playoffs_advanced.csv')  # Steve Nash
shaq_playoffs_advanced = pd.read_csv('datasets/shaq_playoffs_advanced.csv')  # Shaquiel O'Neal

In [10]:
#with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
#    print(lbj_playoffs_adv)

### Merge the playoff totals and advanced stats together.

In [12]:
# Get adv stat fields to merge into playoff_stats.
# Will assume I can use LeBrons columns as a proxy for all players
cols_to_use = lbj_playoffs_advanced.columns.difference(lbj_playoffs_totals.columns)
#type(cols_to_use)

# LeBron
lbj_playoffs_adv2 = lbj_playoffs_advanced[cols_to_use]
lbj_playoffs_adv2['Season'] = lbj_playoffs_advanced.loc[:, 'Season']
lbj_playoffs = pd.merge(lbj_playoffs_totals, lbj_playoffs_adv2, on='Season', how='left')

# Larry Bird
bird_playoffs_advanced2 = bird_playoffs_advanced[cols_to_use]
bird_playoffs_advanced2['Season'] = bird_playoffs_advanced.loc[:, 'Season']
bird_playoffs = pd.merge(bird_playoffs_totals, bird_playoffs_advanced2, on='Season', how='left')

# Magic Johnson
magic_playoffs_advanced2 = magic_playoffs_advanced[cols_to_use]
magic_playoffs_advanced2['Season'] = magic_playoffs_advanced.loc[:, 'Season']
magic_playoffs = pd.merge(magic_playoffs_totals, magic_playoffs_advanced2, on='Season', how='left')

# Kobe Bryant
kobe_playoffs_advanced2 = kobe_playoffs_advanced[cols_to_use]
kobe_playoffs_advanced2['Season'] = kobe_playoffs_advanced.loc[:, 'Season']
kobe_playoffs = pd.merge(kobe_playoffs_totals, kobe_playoffs_advanced2, on='Season', how='left')

# Steph Curry
curry_playoffs_advanced2 = curry_playoffs_advanced[cols_to_use]
curry_playoffs_advanced2['Season'] = curry_playoffs_advanced.loc[:, 'Season']
curry_playoffs = pd.merge(curry_playoffs_totals, curry_playoffs_advanced2, on='Season', how='left')

# Chris Paul
chrispaul_playoffs_advanced2 = chrispaul_playoffs_advanced[cols_to_use]
chrispaul_playoffs_advanced2['Season'] = chrispaul_playoffs_advanced.loc[:, 'Season']
chrispaul_playoffs = pd.merge(chrispaul_playoffs_totals, chrispaul_playoffs_advanced2, on='Season', how='left')

# Kevin Garnett
kg_playoffs_advanced2 = kg_playoffs_advanced[cols_to_use]
kg_playoffs_advanced2['Season'] = kg_playoffs_advanced.loc[:, 'Season']
kg_playoffs = pd.merge(kg_playoffs_totals, kg_playoffs_advanced2, on='Season', how='left')

# Michael Jordan
mj_playoffs_advanced2 = mj_playoffs_advanced[cols_to_use]
mj_playoffs_advanced2['Season'] = mj_playoffs_advanced.loc[:, 'Season']
mj_playoffs = pd.merge(mj_playoffs_totals, mj_playoffs_advanced2, on='Season', how='left')

# James Harden
harden_playoffs_advanced2 = harden_playoffs_advanced[cols_to_use]
harden_playoffs_advanced2['Season'] = harden_playoffs_advanced.loc[:, 'Season']
harden_playoffs = pd.merge(harden_playoffs_totals, harden_playoffs_advanced2, on='Season', how='left')

# Kevin Durant
durant_playoffs_advanced2 = durant_playoffs_advanced[cols_to_use]
durant_playoffs_advanced2['Season'] = durant_playoffs_advanced.loc[:, 'Season']
durant_playoffs = pd.merge(durant_playoffs_totals, durant_playoffs_advanced2, on='Season', how='left')

# Russell Westbrook
westbrook_playoffs_advanced2 = westbrook_playoffs_advanced[cols_to_use]
westbrook_playoffs_advanced2['Season'] = westbrook_playoffs_advanced.loc[:, 'Season']
westbrook_playoffs = pd.merge(westbrook_playoffs_totals, westbrook_playoffs_advanced2, on='Season', how='left')

# Steve Nash
nash_playoffs_advanced2 = nash_playoffs_advanced[cols_to_use]
nash_playoffs_advanced2['Season'] = nash_playoffs_advanced.loc[:, 'Season']
nash_playoffs = pd.merge(nash_playoffs_totals, nash_playoffs_advanced2, on='Season', how='left')

# Shaq
shaq_playoffs_advanced2 = shaq_playoffs_advanced[cols_to_use]
shaq_playoffs_advanced2['Season'] = shaq_playoffs_advanced.loc[:, 'Season']
shaq_playoffs = pd.merge(shaq_playoffs_totals, shaq_playoffs_advanced2, on='Season', how='left')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [13]:
nash_playoffs.columns.values

array(['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', '3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr',
       'OBPM', 'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%',
       'USG%', 'Unnamed: 19', 'Unnamed: 24', 'VORP', 'WS', 'WS/48'],
      dtype=object)

In [14]:
# Add field to signify if playoff games or not
lbj_playoffs['playoffs'] = True
bird_playoffs['playoffs'] = True
magic_playoffs['playoffs'] = True
kobe_playoffs['playoffs'] = True
curry_playoffs['playoffs'] = True
chrispaul_playoffs['playoffs'] = True
kg_playoffs['playoffs'] = True
mj_playoffs['playoffs'] = True
harden_playoffs['playoffs'] = True
durant_playoffs['playoffs'] = True
westbrook_playoffs['playoffs'] = True
nash_playoffs['playoffs'] = True
shaq_playoffs['playoffs'] = True

In [15]:
# Remove empty columns
lbj_playoffs.dropna(axis=1, how='all', inplace=True)
bird_playoffs.dropna(axis=1, how='all', inplace=True)
magic_playoffs.dropna(axis=1, how='all', inplace=True)
kobe_playoffs.dropna(axis=1, how='all', inplace=True)
curry_playoffs.dropna(axis=1, how='all', inplace=True)
chrispaul_playoffs.dropna(axis=1, how='all', inplace=True)
kg_playoffs.dropna(axis=1, how='all', inplace=True)
mj_playoffs.dropna(axis=1, how='all', inplace=True)
harden_playoffs.dropna(axis=1, how='all', inplace=True)
durant_playoffs.dropna(axis=1, how='all', inplace=True)
westbrook_playoffs.dropna(axis=1, how='all', inplace=True)
nash_playoffs.dropna(axis=1, how='all', inplace=True)
shaq_playoffs.dropna(axis=1, how='all', inplace=True)

In [16]:
# Use 'subset=[col_name]' to drop the row with no team entry -- this is the 'career totals' row
lbj_playoffs.dropna(subset=['Tm'], inplace=True)

In [17]:
# Add 'Player' column
lbj_playoffs['Player'] = 'LeBron James'
bird_playoffs['Player'] = 'Larry Bird'
magic_playoffs['Player'] = 'Magic Johnson'
kobe_playoffs['Player'] = 'Kobe Bryant'
curry_playoffs['Player'] = 'Stephen Curry'
chrispaul_playoffs['Player'] = 'Chris Paul'
kg_playoffs['Player'] = 'Kevin Garnett'
mj_playoffs['Player'] = 'Michael Jordan'
harden_playoffs['Player'] = 'James Harden'
durant_playoffs['Player'] = 'Kevin Durant'
westbrook_playoffs['Player'] = 'Russell Westbrook'
nash_playoffs['Player'] = 'Steve Nash'
shaq_playoffs['Player'] = "Shaquille O'Neal"

In [19]:
# Isolate regular season stats for given player, and add the 'playoffs' column

# Lebron James
lbj_reg = stats[stats.Player == 'LeBron James']
lbj_reg['playoffs'] = False

# Larry Bird
bird_reg = stats[stats.Player == 'Larry Bird']
bird_reg['playoffs'] = False

# Magic Johnson
magic_reg = stats[stats.Player == 'Magic Johnson']
magic_reg['playoffs'] = False

# Kobe Bryant
kobe_reg = stats[stats.Player == 'Kobe Bryant']
kobe_reg['playoffs'] = False

# Steph Curry
curry_reg = stats[stats.Player == 'Stephen Curry']
curry_reg['playoffs'] = False

# Chris Paul
chrispaul_reg = stats[stats.Player == 'Chris Paul']
chrispaul_reg['playoffs'] = False

# Kevin Garnett
kg_reg = stats[stats.Player == 'Kevin Garnett']
kg_reg['playoffs'] = False

# Michael Jordan
mj_reg = stats[stats.Player == 'Michael Jordan']
mj_reg['playoffs'] = False

# James Harden
harden_reg = stats[stats.Player == 'James Harden']
harden_reg['playoffs'] = False

# Kevin Durant
durant_reg = stats[stats.Player == 'Kevin Durant']
durant_reg['playoffs'] = False

# Russell Westbrook
westbrook_reg = stats[stats.Player == 'Russell Westbrook']
westbrook_reg['playoffs'] = False

# Steve Nash
nash_reg = stats[stats.Player == 'Steve Nash']
nash_reg['playoffs'] = False

# Shaquille O'Neal
shaq_reg = stats[stats.Player == "Shaquille O'Neal"]
shaq_reg['playoffs'] = False

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

Add a column in the regular season stats for 'Season', and a 'Year' column to the playoffs stats.

In [20]:
#with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
#    print(lbj_reg)
lbj_reg.columns

Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'playoffs'],
      dtype='object')

In [21]:
#with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
#    print(lbj_playoffs)
lbj_playoffs.columns

Index(['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '3PAr',
       'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM', 'ORB%',
       'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP', 'WS',
       'WS/48', 'playoffs', 'Player'],
      dtype='object')

In [22]:
# Any columns in one dataset and not the other?
lbj_playoffs.columns.difference(lbj_reg.columns)

Index(['Lg', 'Season'], dtype='object')

In [26]:
# Just 'Lg', which is the 'league'. Can remove this
#lbj_playoffs.drop('Lg', axis=1, inplace=True)

# Do this after join

### Join the playoff and regular season stats

In [24]:
# Join the regular season and playoff dataframes
#lbj_full = lbj_reg.append(lbj_playoffs, ignore_index=True)

player_dfs = [
       lbj_playoffs, 
       bird_reg, bird_playoffs, 
       magic_reg, magic_playoffs, 
       kobe_reg, kobe_playoffs,
       curry_reg, curry_playoffs, 
       chrispaul_reg, chrispaul_playoffs, 
       kg_reg, kg_playoffs, 
       mj_reg, mj_playoffs, 
       harden_reg, harden_playoffs, 
       durant_reg, durant_playoffs, 
       westbrook_reg, westbrook_playoffs, 
       nash_reg, nash_playoffs, 
       shaq_reg, shaq_playoffs
       ]

join_df = lbj_reg.append(player_dfs, ignore_index=True)

In [30]:
join_df[['Player', 'Season', 'Year', 'playoffs']].head()

,Player,Season,Year,playoffs
0,LeBron James,NaN,2004.0,False
1,LeBron James,NaN,2005.0,False
2,LeBron James,NaN,2006.0,False
3,LeBron James,NaN,2007.0,False
4,LeBron James,NaN,2008.0,False


Fill in all the empty 'Season' and 'Year' entries.

In [183]:
# Test first on data subset
df = join_df[['Player', 'Season', 'Year', 'playoffs']].head(30)

#df = df.fillna(0)

# NOTE: THIS IS A REALLY SLOW(!) SOLUTION -- TRIED TO DO VECTORIZED OPERATION USING FILLNA() BUT HAD SOME ISSUES
def SeasonYears(df):
    for idx, n in df.iterrows():
        if n.playoffs:
            df.Year[idx] = df.Season[idx][:2] + df.Season[idx][-2:] #n.Year - 1 #, str(row.Year)[2:4]
        if not n.playoffs:
            df.Season[idx] = str(int(df.Year[idx] - 1)) + '-' + str(int(df.Year[idx]))[2:4]
    return df
    
#df['Season'] = np.where(df.Season.isnull(), df.Year - 1, df.Season)
#df['Season'] = df['Season']].where(df['Season']].isnull(), 1).fillna(0).astype(int)

#join_df['Season'] = join_df.apply(lambda row: '{}-{}'.format(int(row.Year - 1), str(row.Year)[2:4]), axis=1)
#lbj_playoffs['Year'] = lbj_playoffs.apply(lambda row: float(row.Season[:4]), axis=1)

In [186]:
df.dtypes

Player       object
Season       object
Year        float64
playoffs       bool
dtype: object

Seems to be working ok - apply to full dataset.

In [187]:
playoffs_seaons_full = SeasonYears(join_df)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [192]:
playoffs_seaons_full[['Player','Year','Season', 'playoffs']][120:150]

,Player,Year,Season,playoffs
120,Stephen Curry,2017.0,2016-17,False
121,Stephen Curry,2013.0,2012-13,True
122,Stephen Curry,2014.0,2013-14,True
123,Stephen Curry,2015.0,2014-15,True
124,Stephen Curry,2016.0,2015-16,True
125,Stephen Curry,2017.0,2016-17,True
126,Stephen Curry,2018.0,2017-18,True
127,Chris Paul,2006.0,2005-06,False
128,Chris Paul,2007.0,2006-07,False
129,Chris Paul,2008.0,2007-08,False


### Export this formatted dataset for future use

## Dataset is now ready for charting